### Object Detection for auto annotation
현재 해충에 관련되어 instance segmentation은 수행이 완료된 상태이다. 이제 object detection 모델만 완성되면 auto annotation을 실행할 수 있다.

### Object Detection with bug dataset

In [1]:
import os
import cv2
import json
import shutil
import numpy as np
from ultralytics import YOLO

In [2]:
def calculate_bounding_box_from_coordinates(x1, y1, x2, y2):
    width = x2 - x1
    height = y2 - y1
    
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2
    
    return width, height, center_x, center_y

In [23]:
class_num = {  
    "미국선녀벌레" : 0,
    "썩덩나무노린재" : 1,
    "흰점도둑나방" : 2
    }

classes_dir = "./Data/labels/"
class_names = os.listdir(classes_dir)

origin_image_dir = "./Data/images/"  # 원본 이미지가 담겨있는 dir

image_saved_dir = "./detection_data/images"  # 이미지를 복사하여 저장할 경로
label_saved_dir = "./detection_data/labels"  # yolo label 형식에 맞춰 저장할 경로

for class_ in class_names:
    label_dir = os.path.join(classes_dir, class_)
    image_dir = os.path.join(origin_image_dir, class_)
    label_filenames = os.listdir(label_dir)  # json filenames
    for filename in label_filenames:
        json_path = os.path.join(label_dir, filename)
        with open(json_path, "r", encoding="utf8") as file:
            json_file = json.load(file)

        image_filename = json_file["Meta"]["Image_FileName"]
        image_filepath = os.path.join(image_dir, image_filename)

        image_extension = image_filename[image_filename.rfind("."):]

        image = cv2.imread(image_filepath)
        height, width, _ = image.shape

        speices_name = json_file["Meta"]["Scientific_Name"]
        if speices_name != "Bradysia impatiens":
            annotation_txt = ""
            annotations = json_file["Image_Annotation"]["Annotations"]
            for annotation in annotations:
                instance_name = annotation["Object Name"]
                x_coordinate = [i for i in annotation["X Coordinate"]]
                y_coordinate = [i for i in annotation["Y Coordinate"]]

                x_max = np.max(x_coordinate)
                x_min = np.min(x_coordinate)
                y_max = np.max(y_coordinate)
                y_min = np.min(y_coordinate)

                w, h, x, y = calculate_bounding_box_from_coordinates(x_min, y_min, x_max, y_max)
                coordinates = f"{class_num[instance_name]} {x / width} {y / height} {w / width} {h / height}\n"
                annotation_txt += coordinates
            with open(os.path.join(label_saved_dir, image_filename.replace(image_extension, ".txt")), "w", encoding="utf8") as txt_file:
                txt_file.write(annotation_txt)
            txt_file.close()
            shutil.copyfile(image_filepath, os.path.join(image_saved_dir, image_filename))
        file.close()

In [3]:
import splitfolders

splitfolders.ratio("./detection_data", "./output_dataset", seed=11, ratio=(0.6, 0.2, 0.2))

Copying files: 2330 files [02:40, 14.51 files/s] 


In [4]:
os.environ['WANDB_DISABLED'] = 'true'

def main():
    model = YOLO("./yolov8l.pt", task="detect")
    model.train(
        model = "./yolov8l.pt",
        data = "./bug_detection.yaml",
        imgsz=640,
        epochs=100,
        batch=16,  # batch=-1 -> 장비가 허용하는 범위 내에서 동적으로 설정
        workers=2,
        plots=True,
        device=0,  # gpu 사용
        patience=10,
        translate=0.0,  # 수평 혹은 수직이동
    )

if __name__ == "__main__":
    main()

New https://pypi.org/project/ultralytics/8.2.6 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.5  Python-3.9.12 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=detect, mode=train, model=./yolov8l.pt, data=./bug_detection.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save

train: Scanning D:\CudaTest\segmentation\output_dataset\train\labels... 699 images, 0 backgrounds, 0 corrupt: 100%|██████████| 699/699 [00:08<00:00, 82.41it/s] 

train: WARNING  D:\CudaTest\segmentation\output_dataset\train\images\20221201_150000_2_2984.png: 1 duplicate labels removed
train: WARNING  D:\CudaTest\segmentation\output_dataset\train\images\20221201_150000_3_10639.png: 1 duplicate labels removed
train: WARNING  D:\CudaTest\segmentation\output_dataset\train\images\20221202_130000_3_10634.png: 1 duplicate labels removed
train: WARNING  D:\CudaTest\segmentation\output_dataset\train\images\20221203_120000_3_7869.png: 1 duplicate labels removed
train: WARNING  D:\CudaTest\segmentation\output_dataset\train\images\20221203_170000_3_7484.png: 1 duplicate labels removed
train: New cache created: D:\CudaTest\segmentation\output_dataset\train\labels.cache



val: Scanning D:\CudaTest\segmentation\output_dataset\val\labels... 233 images, 0 backgrounds, 0 corrupt: 100%|██████████| 233/233 [00:02<00:00, 112.79it/s]

val: WARNING  D:\CudaTest\segmentation\output_dataset\val\images\20221201_150000_1_9710.png: 2 duplicate labels removed
val: WARNING  D:\CudaTest\segmentation\output_dataset\val\images\20221201_150000_2_3446.png: 2 duplicate labels removed
val: WARNING  D:\CudaTest\segmentation\output_dataset\val\images\20221203_120000_2_6178.png: 1 duplicate labels removed
val: New cache created: D:\CudaTest\segmentation\output_dataset\val\labels.cache


...

     78/100        11G     0.3836     0.2396     0.8077         20        640: 100%|██████████| 44/44 [00:42<00:00,  1.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:05<00:00,  1.39it/s]

                   all        233        376      0.942      0.958      0.982      0.912
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 68, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



78 epochs completed in 1.144 hours.
Optimizer stripped from c:\Users\admin\Desktop\CudaTest\runs\detect\train\weights\last.pt, 87.7MB
Optimizer stripped from c:\Users\admin\Desktop\CudaTest\runs\detect\train\weights\best.pt, 87.7MB

Validating c:\Users\admin\Desktop\CudaTest\runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.2.5  Python-3.9.12 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 268 layers, 43608921 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:05<00:00,  1.37it/s]


                   all        233        376      0.944      0.972      0.984      0.913
     Metcalfa_pruinosa        233         83          1      0.986      0.995      0.924
     Halyomorpha_halys        233        147      0.947      0.971      0.988      0.891
 Melanchra_persicariae        233        146      0.885      0.959       0.97      0.923
Speed: 0.2ms preprocess, 19.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to c:\Users\admin\Desktop\CudaTest\runs\detect\train


In [3]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(
    data="./output_dataset/test/images", 
    det_model=r"C:\Users\admin\Desktop\CudaTest\runs\detect\train\weights\best.pt", 
    sam_model=r"sam_b.pt", 
    output_dir="./auto_annotation_test"
    )


image 1/233 d:\CudaTest\segmentation\output_dataset\test\images\20220808_113039__718.png: 640x576 1 Metcalfa_pruinosa, 167.8ms
image 2/233 d:\CudaTest\segmentation\output_dataset\test\images\20220808_120039__135.png: 640x576 2 Metcalfa_pruinosas, 20.7ms
image 3/233 d:\CudaTest\segmentation\output_dataset\test\images\20220808_130039__377.png: 640x576 1 Metcalfa_pruinosa, 24.1ms
image 4/233 d:\CudaTest\segmentation\output_dataset\test\images\20220808_143039__366.png: 640x576 2 Metcalfa_pruinosas, 24.6ms
image 5/233 d:\CudaTest\segmentation\output_dataset\test\images\20220808_143039__653.png: 640x576 1 Metcalfa_pruinosa, 24.0ms
image 6/233 d:\CudaTest\segmentation\output_dataset\test\images\20220808_150039__638.png: 640x576 2 Metcalfa_pruinosas, 27.2ms
image 7/233 d:\CudaTest\segmentation\output_dataset\test\images\20220808_150039__737.png: 640x576 2 Metcalfa_pruinosas, 31.2ms
image 8/233 d:\CudaTest\segmentation\output_dataset\test\images\20220808_153039__398.png: 640x576 2 Metcalfa_pru

### Auto Annotation 진행
Auto annotation을 진행한 뒤에, 원본 파일들과 비교한다.

In [21]:
# Auto Annotation 시작
label_dir = "./auto_annotation_test/"
image_dir = "./output_dataset/test/images/"
image_filenames = os.listdir(image_dir)
for filename in image_filenames:
    image_extension = filename[filename.rfind("."):]
    image_path = os.path.join(image_dir, filename)
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    label_path = os.path.join(label_dir, filename.replace(image_extension, ".txt"))
    with open(label_path, "r", encoding="utf8") as txt:
        lines = txt.read()
    line = lines.split("\n")
    coordinates = [i.split(" ") for i in line if i != ""]
    poly_lines = []
    for coordinate in coordinates:
        points = [float(i) for i in coordinate[1:]]
        points_x = [int(points[i]*width) for i in range(0, len(points), 2)]
        points_y = [int(points[i]*height) for i in range(1, len(points), 2)]

        zipped_point = list(zip(points_x, points_y))
        points_list = np.array([list(i) for i in zipped_point], np.int32)

        poly_lines.append(points_list)

    txt.close()
    img = cv2.polylines(image, poly_lines, isClosed=True, color=(0, 255, 0), thickness=4)
    cv2.imwrite(f"./auto_annotation_test/masks/{filename.replace(image_extension, '')}.png", img)

In [31]:
import json

# 원본 파일과 비교
class_dir = "./Data/labels/"
image_dir = "./output_dataset/test/images/"
image_filenames = os.listdir(image_dir)
image_extensions = [i[i.rfind("."):] for i in image_filenames]
# print(set(image_extensions))  # png 확장자의 이미지만 있음을 확인

# 테스트셋은 원천 데이터에서 무작위로 추출된 결과이기에, 작업할 json파일 경로를 따로 저장한다.
worklist = []

for class_ in os.listdir(class_dir):
    label_dir = os.path.join(class_dir, class_)
    label_filenames = os.listdir(label_dir)
    for filename in label_filenames:
        image_filenames_without_extension = [i[:i.rfind(".")] for i in image_filenames]
        label_filename_without_extension = filename.replace(".json", "")

        if label_filename_without_extension in image_filenames_without_extension:
            worklist.append(os.path.join(label_dir, filename))

In [34]:
len(worklist), len(image_filenames)  # 개수 일치

(233, 233)

In [37]:
for image_filename, label_filename in zip(image_filenames, worklist):
    image_path = os.path.join(image_dir, image_filename)
    image = cv2.imread(image_path)

    with open(label_filename, "r", encoding="utf8") as file:
        json_file = json.load(file)
    coordinates = json_file["Image_Annotation"]["Annotations"]

    poly_lines_from_json = []
    for coordinate in coordinates:
        x_coordinate = coordinate["X Coordinate"]
        y_coordinate = coordinate["Y Coordinate"]

        zipped_point = list(zip(x_coordinate, y_coordinate))
        points_list = np.array([list(i) for i in zipped_point], np.int32)
        poly_lines_from_json.append(points_list)

    file.close()
    img = cv2.polylines(image, poly_lines_from_json, isClosed=True, color=(0, 255, 0), thickness=4)
    cv2.imwrite(f"./auto_annotation_test/masks2/{image_filename}", img)